**If you have 28 rows with .xls format not .xlsx**

And the files with .xls format have 28 rows that contian similar information. On top of that the file type is changed to .xlsx format for consistency with other file types

In [139]:
import glob
import pandas as pd
import xlrd
from openpyxl import Workbook

# Directory path where the Excel files are located
directory = r"D:\IHE\Thesis\Temperature\metingen zomer 2023 kopie\Wimmenum L18 & L19" # Replace with the actual directory path

# Get the list of Excel file paths in the directory
file_paths = glob.glob(directory + '/*.xls')  # Modify the file extension if needed

# Iterate over each file path
for file_path in file_paths:
    # Read the .xls file using xlrd
    xls_workbook = xlrd.open_workbook(file_path)
    
    # Create a new .xlsx file
    xlsx_file_path = file_path.replace('.xls', '.xlsx')
    xlsx_workbook = Workbook()

    # Iterate through sheets in the .xls workbook
    for sheet_index in range(xls_workbook.nsheets):
        xls_sheet = xls_workbook.sheet_by_index(sheet_index)
        xlsx_sheet = xlsx_workbook.active if sheet_index == 0 else xlsx_workbook.create_sheet()

        # Copy data from .xls sheet to .xlsx sheet
        for row_index in range(xls_sheet.nrows):
            row_data = xls_sheet.row_values(row_index)
            xlsx_sheet.append(row_data)

    # Save the .xlsx workbook
    xlsx_workbook.save(xlsx_file_path)

    print(f"Converted {file_path} to {xlsx_file_path}")
    
# Get the list of Excel file paths in the directory
file_paths = glob.glob(directory + '/*.xlsx')  # Modify the file extension if needed

# Iterate over each file path
for file_path in file_paths:
    # Read the Excel file into a DataFrame, skipping the first 21 rows and excluding the header row
    df = pd.read_excel(file_path, skiprows=28, header=None)
    
    # Save the modified DataFrame back to the Excel file, excluding the header row
    df.to_excel(file_path, index=False, header=False)


Converted D:\IHE\Thesis\Temperature\metingen zomer 2023 kopie\Wimmenum L18 & L19\L18 Lente 2023 (5).xls to D:\IHE\Thesis\Temperature\metingen zomer 2023 kopie\Wimmenum L18 & L19\L18 Lente 2023 (5).xlsx
Converted D:\IHE\Thesis\Temperature\metingen zomer 2023 kopie\Wimmenum L18 & L19\L18 Zomer 2023 (1).xls to D:\IHE\Thesis\Temperature\metingen zomer 2023 kopie\Wimmenum L18 & L19\L18 Zomer 2023 (1).xlsx
Converted D:\IHE\Thesis\Temperature\metingen zomer 2023 kopie\Wimmenum L18 & L19\L18 Zomer 2023 (2).xls to D:\IHE\Thesis\Temperature\metingen zomer 2023 kopie\Wimmenum L18 & L19\L18 Zomer 2023 (2).xlsx
Converted D:\IHE\Thesis\Temperature\metingen zomer 2023 kopie\Wimmenum L18 & L19\L18 Zomer 2023 (3).xls to D:\IHE\Thesis\Temperature\metingen zomer 2023 kopie\Wimmenum L18 & L19\L18 Zomer 2023 (3).xlsx
Converted D:\IHE\Thesis\Temperature\metingen zomer 2023 kopie\Wimmenum L18 & L19\L18 Zomer 2023 (4).xls to D:\IHE\Thesis\Temperature\metingen zomer 2023 kopie\Wimmenum L18 & L19\L18 Zomer 2023

**Combine excel files in a directory**

Make sure in the raw files the temperature data is stored as number format, as sometimes it is stored as text. This will mess the order of dates in the combined file. **But also change the column name of the converted file (from .xls to match with the .xlsx) by copying the column name that containes the temperature values from the .xlsx file and paste in the converted file (This is only if you get two types of data for one slack, if both of them are .xls renaming is not needed)** 

In [140]:
import pandas as pd
import os

# Specify the directory path containing the files
directory_path = r"D:\IHE\Thesis\Temperature\metingen zomer 2023 kopie\Wimmenum L18 & L19"
directory_path2 = r"D:\IHE\Thesis\Temperature\metingen zomer 2023 kopie\Wimmenum L18 & L19\Output"
# Get a list of all Excel files in the directory
file_list = [file for file in os.listdir(directory_path) if file.endswith(".xlsx")]

# Initialize an empty list to store DataFrames
dfs = []

# Loop through each file and read it into a DataFrame
for file in file_list:
    file_path = os.path.join(directory_path, file)
    df = pd.read_excel(file_path)
    dfs.append(df)

# Concatenate all DataFrames in the list
combined_df = pd.concat(dfs, ignore_index=True)

# Save the combined DataFrame to a new Excel file
output_file = os.path.join(directory_path2, "Combined_Wimm.xlsx")
combined_df.to_excel(output_file, index=False)


**Extract night time temperature**

In [141]:
import pandas as pd

# Read the Excel file into a DataFrame
df = pd.read_excel("D:\IHE\Thesis\Temperature\metingen zomer 2023 kopie\Wimmenum L18 & L19\Output\Combined_Wimm.xlsx")

# Convert 'Time' column to datetime type
df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d %H:%M:%S')

# Define the desired time range
start_time = pd.to_datetime('20:00:00', format='%H:%M:%S').time()
end_time = pd.to_datetime('06:00:00', format='%H:%M:%S').time()

# Filter the DataFrame based on the date and time range
filtered_df = df[
    (df['Time'].dt.time >= start_time) | (df['Time'].dt.time <= end_time)
]

# Print the filtered DataFrame
print(filtered_df)


        No.                Time  Temperature°C  Humidity%
37       38 2023-06-02 20:03:15           17.0        NaN
38       39 2023-06-02 20:08:15           16.0        NaN
39       40 2023-06-02 20:13:15           15.4        NaN
40       41 2023-06-02 20:18:15           15.0        NaN
41       42 2023-06-02 20:23:15           14.7        NaN
...     ...                 ...            ...        ...
25209  3877 2023-08-29 05:35:58           12.6        NaN
25210  3878 2023-08-29 05:40:58           12.5        NaN
25211  3879 2023-08-29 05:45:58           12.5        NaN
25212  3880 2023-08-29 05:50:58           12.4        NaN
25213  3881 2023-08-29 05:55:58           12.4        NaN

[10560 rows x 4 columns]


In [142]:
filtered_df.to_excel(r"D:\IHE\Thesis\Temperature\metingen zomer 2023 kopie\Wimmenum L18 & L19\Output\Noctural_Wimm.xlsx", index=False)
